In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!apt install git-lfs



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
%cd /content/drive/MyDrive/
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/OpenGVLab/InternVL2_5-4B


/content/drive/MyDrive
fatal: destination path 'InternVL2_5-4B' already exists and is not an empty directory.


In [ ]:
!ls -lh /content/drive/MyDrive/InternVL2_5-4B


total 5.1M
-rw------- 1 root root  790 Feb 12 15:07 added_tokens.json
-rw------- 1 root root 3.7K Feb 12 15:07 config.json
-rw------- 1 root root 5.5K Feb 12 15:07 configuration_intern_vit.py
-rw------- 1 root root 3.8K Feb 12 15:07 configuration_internvl_chat.py
-rw------- 1 root root  15K Feb 12 15:07 conversation.py
drwx------ 2 root root 4.0K Feb 12 15:07 examples
-rw------- 1 root root  129 Feb 12 15:07 generation_config.json
-rw------- 1 root root 1.6M Feb 12 15:07 merges.txt
-rw------- 1 root root  135 Feb 12 15:07 model-00001-of-00002.safetensors
-rw------- 1 root root  135 Feb 12 15:07 model-00002-of-00002.safetensors
-rw------- 1 root root  18K Feb 12 15:07 modeling_intern_vit.py
-rw------- 1 root root  16K Feb 12 15:07 modeling_internvl_chat.py
-rw------- 1 root root  71K Feb 12 15:07 model.safetensors.index.json
drwx------ 2 root root 4.0K Feb 12 15:08 models--OpenGVLab--InternVL2_5-4B
-rw------- 1 root root  287 Feb 12 15:07 preprocessor_config.json
-rw------- 1 root root 

In [ ]:
import torch
from transformers import AutoModel, AutoProcessor
from PIL import Image
import torchvision.transforms as transforms

# Define model path
path = "OpenGVLab/InternVL2_5-4B"
local_dir = "/content/drive/MyDrive/InternVL2_5-4B"

# Ensure device compatibility
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32  # Ensure model and input types match

# Load the model
model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

# Load processor (acts as tokenizer)
processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

# Function to preprocess image (Fix: Convert to torch.bfloat16 if using GPU)
def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension & move to device
    return image.to(torch_dtype)  # Ensure correct dtype for model

# Load and preprocess the image
image_path = "/content/drive/MyDrive/P_DOC/HealthID.jpeg"  # Replace with your actual image path
image = preprocess_image(image_path)

# Define text input
text = "Describe the image."

# Fix: Use processor directly for text
inputs = processor(text=[text], return_tensors="pt").to(device)

# Fix: Ensure inputs are converted to model dtype
inputs["pixel_values"] = image

# Run model inference
with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

print("Assistant:", response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-4B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-4B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: The image is of an Aadhaar card issued by the National Health Authority (NHA). The cardholder's name is Madhuri, and the Aadhaar number is 91-2262-227-4311. The card is for a female, and the date of birth is 05-12-2004. The Aadhaar number is 97059985. The card also includes a QR code and a hologram for security. There are instructions on the card regarding the use of the Aadhaar card for various purposes, including digital health services. The card is issued by the National Health Authority (NHA) in India.


In [ ]:
import torch
import time
from transformers import AutoModel, AutoProcessor
from PIL import Image
import torchvision.transforms as transforms

path = "OpenGVLab/InternVL2_5-4B"
local_dir = "/content/drive/MyDrive/InternVL2_5-4B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

torch.backends.cudnn.benchmark = True  # Optimize CUDA performance

model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

image_path = "/content/sample.jpeg"
image = preprocess_image(image_path)

text = "You are a document ID API that extracts and sends details from driving licenses in JSON format. Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, License Number, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers. If the image is upside down, turned left or right, fix it and get the details. If the image is missing details please set the value as empty string like this \"\", If the image does not look like an ID, return an empty JSON. Only reply in JSON."

inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

start_time = time.time()

with torch.no_grad():
    response = model.chat(
        processor,
        inputs["pixel_values"],
        text,
        generation_config={
            "max_new_tokens": 512,
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
            "repetition_penalty": 1.1
        }
    )

end_time = time.time()
inference_time = end_time - start_time

print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: ```json
{
  "Document Type": "Driver's Licence",
  "Issuing Authority": "Western Australia",
  "Name": "Shaun Robert Paul",
  "Date of Birth": "18 Apr 2027",
  "Gender": "",
  "License Number": "",
  "Address or Location": "Ellenbrook, WA 6059",
  "Document Number": "07101963",
  "Expiry Date": "",
  "Nationality": "",
  "Other Identifiers": ""
}
```
Inference Time: 7.4505 seconds


In [ ]:
import torch
import time
import os
from transformers import AutoModel, AutoProcessor
from PIL import Image
import torchvision.transforms as transforms
torch.cuda.empty_cache()

# Set environment variable to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Model and device setup
path = "OpenGVLab/InternVL2_5-4B"
local_dir = "/content/drive/MyDrive/InternVL2_5-4B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# Load model with optimized settings
model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

# Load processor
processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

# Image preprocessing
def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

# Path to image
image_path = "/content/sample.jpeg"
image = preprocess_image(image_path)

# Text prompt
text = "You are a document ID API that extracts and sends details from driving licenses in JSON format . Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, License Number, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers.If the image is upside down,turned left or right ,  fix it and get the details. If the image is missing details please set the value as empty string like this  \"\" , If the image is not looks like ID means send an empty JSON. Only reply in JSON"

# Prepare inputs
inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

# Efficient inference parameters for generation
generation_config = {
    # Core parameters
    "max_new_tokens": 512,               # Maximum number of tokens to generate
    "temperature": 0.7,                  # Controls randomness (lower = more deterministic)
    "top_p": 0.9,                        # Nucleus sampling (top-p)
    "top_k": 50,                         # Limits sampling to top-k tokens
    "repetition_penalty": 1.1,           # Explicitly penalizes repeated tokens
    "stop": ["\n", "###"],               # Stops generation when these tokens are encountered

               # Apply AWQ quantization to reduce model weights
}

# Measure inference time
start_time = time.time()

with torch.no_grad():
    response = model.chat(
        processor,
        inputs["pixel_values"],
        text,
        generation_config=generation_config  # Pass efficient inference parameters
    )

end_time = time.time()
inference_time = end_time - start_time

# Output results
print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-4B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-4B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


ValueError: The following `model_kwargs` are not used by the model: ['stop'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
import time
from transformers import AutoModel, AutoProcessor
from PIL import Image, ImageFile
import torchvision.transforms as transforms

ImageFile.LOAD_TRUNCATED_IMAGES = True

path = "OpenGVLab/InternVL2-2B"
local_dir = "/content/drive/MyDrive/InternVL2-2B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    try:
        image = Image.open(image_path).convert("RGB")
    except OSError:
        print(f"Error loading image: {image_path}. Check if the file is corrupted.")
        return None
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

image_path = "/content/mukundan.jpeg"
image = preprocess_image(image_path)

if image is None:
    exit()

text = "You are a document ID API that extracts and sends details from driving licenses in JSON format . Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers.If the image is upside down,turned left or right ,  fix it and get the details. If the image is missing details please set the value as empty string like this  \"\" , If the image is not looks like ID means send an empty JSON. Only reply in JSON"

inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

start_time = time.time()

with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

end_time = time.time()
inference_time = end_time - start_time

print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")


Assistant: ```json
{
  "Document Type": "Tamil Nadu Driving Licence",
  "Issuing Authority": "Union of India",
  "Name": "MUKUNTHAN KALAISELVAN",
  "Date of Birth": "02-05-2001",
  "Gender": "Male",
  "Address": "O+",
  "Document Number": "TN85 2020000198",
  "Expiry Date": "01-05-2041",
  "Nationality": "Indian",
  "Other Identifiers": ""
}
```
Inference Time: 5.1139 seconds
